In [ ]:
# default_exp predefined_problems.test_data


In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


# Pre-defined Problems

This is for test purpose. Please don't use them. 

## Imports

In [ ]:
# export
import numpy as np
from typing import List
import inspect

from m3tl.preproc_decorator import preprocessing_fn
from m3tl.utils import get_or_make_label_encoder


## Utils

In [ ]:
# export
def _gen_fake_text(tokenized=False) -> List[str]:
    if not tokenized:
        return ['this is a test' for _ in range(10)]
    return ['this is a test'.split() for _ in range(10)]


def _gen_fake_array() -> np.ndarray:
    return [np.random.uniform(size=(10)) for _ in range(10)]


def _gen_fake_cate() -> str:
    return [0 for _ in range(10)]


def _gen_label(label_format='string'):
    if label_format == 'string':
        return ['a' for _ in range(5)] + ['b' for _ in range(5)]

    if label_format == 'float':
        return [0.1 for _ in range(10)]

    if label_format == 'seq_tag_string':
        return ['a b c d'.split() for _ in range(10)]

    if label_format == 'array':
        return [np.random.uniform(size=(10)) for _ in range(10)]

    raise ValueError('Unknown label format: {}'.format(label_format))


def generate_fake_data(output_format='list_tuple', label_format='string') -> dict:
    if output_format == 'list_tuple':
        return _gen_fake_text(tokenized=(label_format == 'seq_tag_string')), _gen_label(label_format=label_format)

    if output_format == 'gen_list_tuple':
        return zip(
            _gen_fake_text(tokenized=(label_format == 'seq_tag_string')), _gen_label(
                label_format=label_format)
        )

    if output_format == 'dict_tuple':
        return ({
            'inputs_text': _gen_fake_text(),
            'inputs_array': _gen_fake_array(),
            'inputs_cate': _gen_fake_cate(),
            'inputs_cate_modal_type': ['category' for _ in range(10)],
            'inputs_cate_modal_info': [1 for _ in range(10)]
        },
        _gen_label(label_format=label_format))


    if output_format == 'gen_dict_tuple':
        dict_list = []
        for text, arr, cate, label in zip(
                _gen_fake_text(),
                _gen_fake_array(),
                _gen_fake_cate(),
                _gen_label(label_format=label_format)):
            dict_list.append(({
                'inputs_text': text,
                'inputs_array': arr,
                'inputs_cate': cate,
                'inputs_cate_modal_type': 'category',
                'inputs_cate_modal_info': 1,
            }, label))
        return iter(dict_list)

    dict_list = []
    for record_id, (text, arr, cate, label) in enumerate(
            zip(
                _gen_fake_text(),
                _gen_fake_array(),
                _gen_fake_cate(),
                _gen_label(label_format=label_format)
            )):
        dict_list.append({
            'inputs_record_id': record_id,
            'inputs_text': text,
            'inputs_array': arr,
            'inputs_cate': cate,
            'inputs_cate_modal_type': 'category',
            'inputs_cate_modal_info': 1,
            'labels': label
        })

    if output_format == 'dict':
        return dict_list

    if output_format == 'gen_dict':
        return iter(dict_list)

    if output_format == 'rdd':
        from pyspark import SparkContext
        sc: SparkContext = SparkContext.getOrCreate()
        return sc.parallelize(dict_list)


def create_fake_label_encoder(params, problem: str, mode: str, label_format='string'):
    label_list = _gen_label(label_format=label_format)
    get_or_make_label_encoder(
        params=params, problem=problem, mode=mode, label_list=label_list, overwrite=True)


## Local problems

In [ ]:
# export

def get_fake_contrastive_learning_fn(file_path):
    @preprocessing_fn
    def fake_contrastive_learning(params, mode):
        """Just a test problem to test multiproblem support

        Arguments:
            params {Params} -- params
            mode {mode} -- mode
        """
        return generate_fake_data(output_format='list_tuple', label_format='string')

    return fake_contrastive_learning

def get_weibo_fake_cls_fn(file_path):
    @preprocessing_fn
    def weibo_fake_cls(params, mode):
        """Just a test problem to test multiproblem support

        Arguments:
            params {Params} -- params
            mode {mode} -- mode
        """
        return generate_fake_data(output_format='list_tuple', label_format='string')

    return weibo_fake_cls


def get_weibo_fake_ner_fn(file_path):
    @preprocessing_fn
    def weibo_fake_ner(params, mode):
        return generate_fake_data(output_format='list_tuple', label_format='seq_tag_string')
    return weibo_fake_ner


def get_weibo_pretrain_fn(file_path):
    @preprocessing_fn
    def weibo_pretrain(params, mode):
        return _gen_fake_text()
    return weibo_pretrain


def get_weibo_fake_multi_cls_fn(file_path):
    @preprocessing_fn
    def weibo_fake_multi_cls(params, mode):
        return generate_fake_data(output_format='dict_tuple', label_format='seq_tag_string')
    return weibo_fake_multi_cls


def get_weibo_masklm(file_path):
    @preprocessing_fn
    def weibo_masklm(params, mode):
        # Gen has to manual call get or make label encoder!
        create_fake_label_encoder(params=params, problem=inspect.currentframe(
        ).f_code.co_name, mode=mode, label_format='string')
        return generate_fake_data(output_format='gen_list_tuple', label_format='string')
    return weibo_masklm


def get_weibo_premask_mlm(file_path):
    @preprocessing_fn
    def weibo_premask_mlm(params, mode):
        create_fake_label_encoder(params=params, problem=inspect.currentframe(
        ).f_code.co_name, mode=mode, label_format='string')
        return generate_fake_data(output_format='gen_list_tuple', label_format='string')
    return weibo_premask_mlm


def get_weibo_fake_regression(file_path):
    @preprocessing_fn
    def weibo_fake_regression(params, mode):
        return generate_fake_data(output_format='gen_dict', label_format='float')
    return weibo_fake_regression


def get_weibo_fake_vector_fit(file_path):
    @preprocessing_fn
    def weibo_fake_vector_fit(params, mode):
        create_fake_label_encoder(params=params, problem=inspect.currentframe(
        ).f_code.co_name, mode=mode, label_format='array')
        return generate_fake_data(output_format='gen_list_tuple', label_format='array')
    return weibo_fake_vector_fit


In [ ]:
# hide
from m3tl.test_base import TestBase
from m3tl.special_tokens import TRAIN
tb = TestBase()
fn = get_weibo_fake_multi_cls_fn('')
res = fn(params=tb.params, mode=TRAIN)


Adding new problem weibo_fake_ner, problem type: seq_tag
Adding new problem weibo_cws, problem type: seq_tag
Adding new problem weibo_fake_multi_cls, problem type: multi_cls
Adding new problem weibo_fake_cls, problem type: cls
Adding new problem weibo_masklm, problem type: masklm
Adding new problem weibo_fake_regression, problem type: regression
Adding new problem weibo_fake_vector_fit, problem type: vector_fit
Adding new problem weibo_premask_mlm, problem type: premask_mlm


In [ ]:
generate_fake_data(output_format='dict_tuple', label_format='seq_tag_string')


({'text': ['this is a test',
   'this is a test',
   'this is a test',
   'this is a test',
   'this is a test',
   'this is a test',
   'this is a test',
   'this is a test',
   'this is a test',
   'this is a test'],
  'array': [array([0.49095954, 0.20541639, 0.49262084, 0.87451381, 0.02458258,
          0.65257202, 0.98258836, 0.58114124, 0.58505677, 0.15806585]),
   array([0.44947885, 0.03171102, 0.62730992, 0.7299056 , 0.04007493,
          0.65133139, 0.60520342, 0.15861278, 0.29668022, 0.33993872]),
   array([0.2731397 , 0.01539462, 0.53834935, 0.32880503, 0.53450392,
          0.76220641, 0.44569313, 0.78168369, 0.54579642, 0.71716377]),
   array([0.12633738, 0.75136335, 0.32964124, 0.64901069, 0.09048094,
          0.96317688, 0.47199601, 0.45125637, 0.53099414, 0.51856527]),
   array([0.75426461, 0.76001104, 0.22953306, 0.9394916 , 0.37764847,
          0.50555047, 0.93132036, 0.47237353, 0.48608123, 0.0177382 ]),
   array([0.4976838 , 0.06866748, 0.37725227, 0.12630836, 0.23

## Pyspark Problems

In [ ]:
# export

@preprocessing_fn
def pyspark_fake_cls(params, mode):
    """Just a test problem to test multiproblem support

    Arguments:
        params {Params} -- params
        mode {mode} -- mode
    """
    create_fake_label_encoder(params=params, problem=inspect.currentframe(
        ).f_code.co_name, mode=mode, label_format='string')
    return generate_fake_data(output_format='rdd', label_format='string')


@preprocessing_fn
def pyspark_fake_seq_tag(params, mode):
    create_fake_label_encoder(params=params, problem=inspect.currentframe(
        ).f_code.co_name, mode=mode, label_format='seq_tag_string')
    return generate_fake_data(output_format='rdd', label_format='seq_tag_string')


@preprocessing_fn
def pyspark_fake_multi_cls(params, mode):
    create_fake_label_encoder(params=params, problem=inspect.currentframe(
        ).f_code.co_name, mode=mode, label_format='seq_tag_string')
    return generate_fake_data(output_format='rdd', label_format='seq_tag_string')
